# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Adtividad Semana 5

### **Vectores Embebidos Pre-entrenados: Fasttext**

#### **Nombres y matrículas de los integrantes del equipo:**



*   Elemento de lista
*   Elemento de lista



In [1]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:

import pandas as pd  
import numpy as np

import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords') #Para descargar las stopwords en caso de que aún no estén descargadas

import re
import string

import contractions #Se importa esta libería para expander las contracciones

from nltk.corpus import wordnet
#nltk.download("wordnet")
#nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.stem import RegexpStemmer 

from sklearn.model_selection import train_test_split

from collections import Counter

import fasttext
import fasttext.util

import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

##**Pregunta - 1:**



Descarga los 3 archivos de Canvas y genera un nuevo DataFrame de Pandas con ellos.

**Llama simplemente "df" a dicho DataFrame.**

Los archivos los encuentras en Canvas: amazon5.txt, imdb5.txt, yelp5.txt.



In [2]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

# En el archivo de imdb5 las etiquetas están separadas del comentario por 3 o más espacios en blano
# Por lo tanto se utiliza un regex como delimitador, con la engine "python" la cual sí soporta regex como delimitador

dfa = pd.read_csv('amazon5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')
dfi = pd.read_csv('imdb5.txt', delimiter=r"\s{3}", names=['review','label'], header=None, encoding='utf-8', engine='python')
dfy = pd.read_csv('yelp5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')

# En el dfi, las etiquetas están como float, hay que convertirlas a int


df = pd.concat([dfa, dfi, dfy], ignore_index=True)

print('Total de registros de Amazon:',dfa.shape)
print('Total de registros de IMBD:',dfi.shape)
print('Total de registros de Yelp:',dfy.shape)

print('Total de registros df:',df.shape)

# *********** Aquí termina la sección de agregar código *************


Total de registros de Amazon: (1000, 2)
Total de registros de IMBD: (1000, 2)
Total de registros de Yelp: (1000, 2)
Total de registros df: (3000, 2)


In [3]:
# Verifiquemos la información del DataFrame:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  3000 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [4]:
# Y veamos sus primeros registros:

display(df.head())

# corroboramos que tengamos 50% de comentarios positivos y negativos

display(df['label'].value_counts()) 


,review,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


0    1500
1    1500
Name: label, dtype: int64

## **Pregunta - 2:**

Realiza el proceso de limpieza.

Aplica el preprocesamiento que consideres adecuado, sin embargo, deberás aplicar necesariamente alguna de las técnicas de lematización.











In [5]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

# Consideremos la siguiente lista de palabras asociada a negaciones en inglés:
negwords = [ 'no', 'nor', 'not', 'ain', 'aren', "aren't", 'don', "don't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# Excluye todas las palabras de la lista "negwords" de la lista predeterminada de "stopwords", para definir un nuevo conjunto llamado mystopwords
mystopwords = [word for word in stopwords.words('english') if word not in negwords]

 # Separamos la información: 
 #     La "X" serán los datos de entrada, los comentarios.
 #     La "Y" será la variable de salida, la evaluación.
 # Ambos, X y Y son "Series" 

X = df.review     # Serie de strings
Y = df.label      # Serie de enteros 0s y 1s


def clean_tok(doc):

    # Se convierte todo a minúsculas
  
    clean_tok = doc.lower()
    

    # Se dejan solamente caracteres alfabécticos y apóstrofe (y se eliminan espacios dobles si los hay)
    
    clean_tok = re.sub(r'[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑ\']', ' ', clean_tok)
    
    clean_tok = re.sub(r'\s{2,}', ' ', clean_tok.strip())

        
    # Se lleva a cabo una primer tokenización
       
    clean_tok = clean_tok.split()
    
    
    # Se expanden las contracciones si existen (de: "couldn't" a:  "could not")
    # Se requiere tener instalado el módulo contractions con pip install contractions
    
    clean_tok = [contractions.fix(expanded_word) for expanded_word in clean_tok]
    
    # se remueven las apóstrofes si aún existen despúes de expander las contracciones
        
    clean_tok = [re.sub(r"[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑ]", " ", word) for word in clean_tok]
    
    #Debido a que los tokens que aún tenían apóstrofe quedan como 2 palabras separadas, cada palabra debe ser un token
    # Se lleva a cabo una segunda tokenización
    
    tokens = []
    
    for token in clean_tok:
        token_list = token.split()
        for tkns in token_list:
            tokens.append(tkns)
           
    #No se remueven stopwords ya que para estos modelos pueden alterar el significado
    #https://stackoverflow.com/questions/37325914/should-i-remove-stopwords-when-feed-sentence-to-rnn
    ''' 
    # Se eliminan stopwords
    
    tokens = [word for word in tokens if word not in mystopwords]
    
    # Se consideran solamente tokens de longitud mayor a 1

    tokens = [word for word in tokens if len(word) > 1]
    '''

    return tokens

Xclean_tok = [clean_tok(x) for x in X] 


lemmatizer = WordNetLemmatizer()

pos_tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}


def clean_doc(doc):
    
    # Se aplica la lematización utilizando el diccionario pos_tag_dict
    # para identificar pronombres, adejtivos, verbos y adverbios
    
    # La funcion POS de NLTK requiere una oración de entrada para identificar si 
    # las palabras son pronombres, adejtivos, verbos y adverbios, por lo que se
    # juntan los tokens nuevamente
    
    '''
    if tipo == "token":
        sentence = " ".join(doc)
    else:
        sentence = doc
    '''
    sentence = " ".join(doc)
    
    pos_list_of_tuples = nltk.pos_tag(nltk.word_tokenize(sentence))
    new_sentence_words = []
    for word_idx, word in enumerate(nltk.word_tokenize(sentence)):
        nltk_word_pos = pos_list_of_tuples[word_idx][1]
        wordnet_word_pos = pos_tag_dict.get(nltk_word_pos[0].upper(), None)
        if wordnet_word_pos is not None:
            # print("found: " + wordnet_word_pos + " for: " + word) # para ver qué palabras reconoció
            new_word = lemmatizer.lemmatize(word, wordnet_word_pos)
        else:
            # print("Not found for: " + word) # para ver qué palabras no reconoció
            new_word = lemmatizer.lemmatize(word)

        new_sentence_words.append(new_word)

    tokens = new_sentence_words
    
    #Se remueven los tokens que quedaron con una sola letra 
    
    #tokens = [word for word in tokens if len(word) > 1]
        
    return tokens

Xclean = [clean_doc(x) for x in Xclean_tok] 


# *********** Aquí termina la sección de agregar código *************

In [6]:
# Despleguemos los primeros comentarios después de tu proceso de limpieza:

for x,review in zip(Xclean[0:50], X[0:50]):
  print("--------------------------------------------------------------")  
  print(x)
  print(review)

--------------------------------------------------------------
['so', 'there', 'be', 'no', 'way', 'for', 'me', 'to', 'plug', 'it', 'in', 'here', 'in', 'the', 'u', 'unless', 'i', 'go', 'by', 'a', 'converter']
So there is no way for me to plug it in here in the US unless I go by a converter.
--------------------------------------------------------------
['good', 'case', 'excellent', 'value']
Good case, Excellent value.
--------------------------------------------------------------
['great', 'for', 'the', 'jawbone']
Great for the jawbone.
--------------------------------------------------------------
['tie', 'to', 'charger', 'for', 'conversation', 'last', 'more', 'than', 'minute', 'major', 'problem']
Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!
--------------------------------------------------------------
['the', 'mic', 'be', 'great']
The mic is great.
--------------------------------------------------------------
['i', 'have', 'to', 'jiggle', 'the', 'p

# **Pregunta - 3:**



Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente.

In [7]:

# ************* Inicia la sección de agregar código:*****************************

x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=1) 
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=17)



# *********** Termina la sección de agregar código *************


# verificemos las dimensiones obtenidas:
print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


#**Pregunta - 4:**



Construye tu vocabulario a continuación


In [8]:
# a.	Usa el conjunto de entrenamiento para generar tu vocabulario
#     con un tamaño que consideres adecuado:


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

midiccionario = Counter()    

for k in range(len(x_train)):
    midiccionario.update(x_train[k])
    
min_freq = 3 #se probó con 2,3,4,5,6,7,8 ...los resultados de Accuracy fuero prácticamente los mismos

midicc = {key:val for key, val in midiccionario.items() if val >= min_freq}


# *********** Aquí termina la sección de agregar código *************

In [9]:
# b.	Indica el tamaño del vocabulario generado.

print('Longitud del vocabulario generado:')


# ******* Inicia la sección de agregar código: ***********


print(len(midicc))  

print(list(midicc.items())[0:50])


# *********** Aquí termina la sección de agregar código *************

Longitud del vocabulario generado:
1011
[('his', 33), ('star', 18), ('do', 226), ('not', 437), ('much', 39), ('well', 68), ('with', 201), ('people', 23), ('like', 100), ('and', 759), ('ed', 3), ('just', 87), ('waste', 32), ('i', 717), ('have', 333), ('the', 1357), ('special', 12), ('it', 593), ('suck', 28), ('pay', 9), ('bill', 3), ('but', 145), ('tip', 5), ('because', 39), ('felt', 11), ('server', 15), ('a', 714), ('terrible', 23), ('job', 13), ('how', 36), ('can', 73), ('you', 162), ('call', 19), ('yourself', 3), ('if', 75), ('cook', 6), ('steak', 15), ('understand', 8), ('however', 19), ('keypad', 3), ('be', 1495), ('so', 124), ('tinny', 3), ('that', 235), ('sometimes', 6), ('wrong', 7), ('button', 6), ('highly', 18), ('recommend', 48), ('this', 452)]


c.	¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el vocabulario?


### ++++++++ Inicia la sección de agregar texto: +++++++++++

Porque los conjutnos de validación y prueba se utilizan para ver cómo se comporta el modelo frente a tokens/palabras/reviews que nunca ha visto antes.

Si incluyeramos tokens/palabras/reviews del conjutno de validación y/o prueba el modelo quedaría sobreentrenado ya que se estaría ajsutando perfectamente a todos los tokens/palabras/reviews que se le están ingresando tanto del conjunto de entrenamiento como los conjuntos de validación y prueba.

La idea en general de crear conjuntos de entrenamiento, validación y prueba es para entrenar al modelo con datos del conjunt de entrenamiento y utilizar los datos de los conjuntos de validación y prueba como datos "nuevos" que nunca antes habían sido vistos por el modelo.


### ++++++++ Termina la sección de agregar texto: +++++++++++


In [10]:
# d.	Con el vocabulario generado, filtra los conjuntos de entrenamiento,
#     validación y prueba para que todos los comentarios usen solamente las
#     palabras de este vocabulario.

#     Llamar train_x, val_x y test_x a estos tres conjuntos.


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


train_x = []
for ss in x_train:
  train_x.append([w for w in ss if w in midicc])

val_x = []
for ss in x_val:
  val_x.append([w for w in ss if w in midicc])

test_x = []
for ss in x_test:
  test_x.append([w for w in ss if w in midicc])


# *********** Aquí termina la sección de agregar código *************


In [11]:
# Vemos el resultado de los primeros comentarios del conjunto de entrenamiento:
i = 0
for ss in train_x[0:5]:
  print("---------------------------------------------------------------------")     
  print(ss)
  indice_df = y_train.index[i]
  review_df = df['review'][indice_df]
  print(review_df)
  i = i + 1

---------------------------------------------------------------------
['his', 'star', 'do', 'not', 'much', 'well', 'with', 'people', 'like', 'and', 'ed', 'just', 'waste']
His co-stars don't fare much better, with people like Morgan Freeman, Jonah Hill, and Ed Helms just wasted.
---------------------------------------------------------------------
['i', 'have', 'the', 'special', 'and', 'it', 'suck']
Tonight I had the Elk Filet special...and it sucked.
---------------------------------------------------------------------
['i', 'pay', 'the', 'bill', 'but', 'do', 'not', 'tip', 'because', 'i', 'felt', 'the', 'server', 'do', 'a', 'terrible', 'job']
I paid the bill but did not tip because I felt the server did a terrible job.
---------------------------------------------------------------------
['how', 'can', 'you', 'call', 'yourself', 'a', 'if', 'you', 'can', 'not', 'cook', 'a', 'steak', 'i', 'do', 'not', 'understand']
How can you call yourself a steakhouse if you can't properly cook a steak

#**Pregunta - 5:**


a. Incluye una tabla comparativa de pros y contras entre los modelos FastText, word2vec de Google y Glove de Stanford.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

# INSERTAR TABLA AQUÍ

### ++++++++ Termina la sección de agregar texto: +++++++++++


#**Pregunta - 6:**

Utiliza el modelo FastText de vectores embebidos pre-entrenados de dimensión 300 para generar un nuevo diccionario clave-valor, donde la “clave” será cada token o palabra de tu vocabulario y el “valor” será su vector embebido de dimensión 300.

Este diccionario deberá ser del mismo tamaño que el vocabulario previo que hayas construido previamente.

Es recomendable que una vez que generes el nuevo vocabulario de vectores embebidos, guardes dicho diccionario en un archivo.

Recuerda borrar la variable donde descargaste los 2 millones de vectores embebidos Fasttext.



In [12]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

# Para generar el diccionario se deben quitar las comillas para ejecutar el código
'''
fasttext.util.download_model('en', if_exists='ignore')  # English

ft = fasttext.load_model('cc.en.300.bin')

midicc_kv = {}

for k,w in enumerate(midicc):
    tmp = ft.get_word_vector(w)
    midicc_kv.update({w:tmp})
    
del ft

#Se guarda el diccionario en un archivo serializado
with open('dict_300_ft.pkl', 'wb') as f:
    pickle.dump(midicc_kv, f)
'''

#Se carga el diccionario deserialiando el archivo
with open('dict_300_ft.pkl', 'rb') as f:
    midicc_kv = pickle.load(f)

# *********** Aquí termina la sección de agregar código *************

# **Pregunta - 7:**



Generamos los vectores embebidos a paertir de los conjuntos de entrenamiento, validación y preuba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.

In [13]:
#display(type(train_x))
#display(len(train_x))
vectors = [midicc_kv[keyword] for keyword in train_x[0]]
#vectors = np.array(vectors)

display(vectors)
display(len(vectors[0]))

df_train_x = pd.DataFrame(vectors).transpose() 

display(df_train_x)

df_train_x_mean = df_train_x.mean(axis=1)

display(df_train_x_mean)

df_train_x_mean_ls = df_train_x_mean.tolist()

display(df_train_x_mean_ls)


#los append se hacen  con listas y luego todo se convierte a array
#https://stackoverflow.com/questions/31250129/python-numpy-array-of-numpy-arrays
'''
trainEmb = np.empty()
i = 0
for review in train_x:
    vectors = [midicc_kv[keyword] for keyword in review]
    vectors = np.array(vectors)
    trainEmb.apppend = vectors
    i = i + 1
    
trainEmb[0:5]
'''

[array([ 5.88246584e-02,  2.03970727e-02, -5.95178604e-02,  1.93353534e-01,
        -1.19364768e-01, -5.35052568e-02,  6.21801987e-03, -5.49606085e-02,
         2.55104415e-02,  6.23831488e-02, -8.53093117e-02,  1.01482645e-01,
        -1.19008787e-01, -5.49450330e-02, -9.38263685e-02,  4.43248600e-02,
        -6.48833066e-03, -2.02912223e-02, -1.70241803e-01, -1.02215663e-01,
        -7.96141475e-02,  1.73895732e-01,  2.78468039e-02, -2.57672481e-02,
         1.10301629e-01,  4.20892313e-02,  6.56428933e-02,  1.15610119e-02,
        -4.34568562e-02,  7.75979832e-02,  4.21597715e-03, -1.98223516e-02,
         9.36607346e-02, -1.74132645e-01,  2.68492289e-03, -9.42280143e-02,
        -3.06592360e-02,  9.98567268e-02,  6.47599176e-02, -3.13267745e-02,
         7.68613443e-03,  1.22243203e-01, -2.89970413e-02,  6.35845885e-02,
        -2.04018280e-02, -5.35552949e-03, -6.30449131e-03, -8.62761587e-02,
        -2.00372249e-01,  6.72392547e-02,  7.74944425e-02,  2.41032884e-01,
        -1.0

300

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.058825,-0.286828,-0.034584,-0.063500,0.002116,-0.026274,0.014873,-0.008569,0.019926,0.008239,-0.119590,-0.001510,-0.102139
1,0.020397,0.115124,-0.146392,0.015611,0.070594,-0.065504,-0.115201,-0.033164,0.040573,-0.089903,-0.246485,0.067440,0.102705
2,-0.059518,0.053821,0.059775,-0.013626,0.063566,-0.080330,-0.024813,-0.015297,0.074788,0.026525,-0.446496,0.044536,0.060051
3,0.193354,0.062680,-0.042675,0.247374,-0.002534,0.073466,0.000256,0.051420,0.045387,-0.008554,-0.092834,0.177130,0.100049
4,-0.119365,0.024713,-0.258576,-0.091397,-0.057254,-0.077838,-0.057487,-0.110843,-0.102027,-0.060940,-0.313323,-0.077358,-0.074666
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,-0.038776,0.018275,0.243029,0.076358,0.018939,-0.010485,0.007355,0.034164,0.059033,-0.038051,-0.401446,0.013058,0.036497
296,-0.150690,-0.046839,0.147975,0.022759,-0.039240,-0.047981,-0.031811,0.022906,-0.019528,-0.021103,0.455509,0.017519,-0.088142
297,0.366335,0.056705,-0.053481,0.193799,0.142228,0.136594,-0.026437,0.086877,0.048325,-0.011591,0.434995,0.109533,0.063832
298,-0.030677,-0.005086,0.046709,-0.001168,-0.023257,-0.064167,-0.030942,0.002911,-0.005253,-0.041129,0.174733,-0.026507,0.049242


0     -0.041463
1     -0.020324
2     -0.019771
3      0.061886
4     -0.105874
         ...   
295    0.001381
296    0.017026
297    0.119055
298    0.003493
299   -0.014699
Length: 300, dtype: float32

[-0.04146265983581543,
 -0.020323537290096283,
 -0.01977059245109558,
 0.061886195093393326,
 -0.10587381571531296,
 -0.0225331149995327,
 -0.006103633902966976,
 -0.02595851756632328,
 0.0017375997267663479,
 -0.011520195752382278,
 -0.007658916525542736,
 -0.004932303912937641,
 -0.004894849844276905,
 0.01221115980297327,
 0.016242222860455513,
 0.04248296469449997,
 0.030449997633695602,
 -0.019237222149968147,
 -0.04756459221243858,
 0.06718982011079788,
 -0.001632271334528923,
 0.015085438266396523,
 0.03744577616453171,
 0.019466442987322807,
 -0.017613625153899193,
 -0.043225374072790146,
 0.006266183685511351,
 -0.010233552195131779,
 0.015095055103302002,
 0.10700469464063644,
 -0.0002677867596503347,
 -0.025742530822753906,
 0.01951107569038868,
 -0.004396925680339336,
 -0.002798996400088072,
 -0.010590349324047565,
 -0.0060362434014678,
 0.04868496209383011,
 0.006336390506476164,
 -0.01311589777469635,
 0.02384946495294571,
 -0.021697502583265305,
 -0.0374029278755188,
 0.

'\ntrainEmb = np.empty()\ni = 0\nfor review in train_x:\n    vectors = [midicc_kv[keyword] for keyword in review]\n    vectors = np.array(vectors)\n    trainEmb.apppend = vectors\n    i = i + 1\n    \ntrainEmb[0:5]\n'

In [14]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

n = 300 # cada vector tienen 300 coordenadas. Cuando haya un documento sin tokens se llenara con una lista de 300 ceros

#Definimos una función que itere sobre cada documento del dataset,
#Para cada documento obtenga la lista de vectores de cada token desde el diccionario
#Que convierta la lista de vectores en un dataframe, para calcular el promedio de las 300 coords
#Que convierta el dataframe nuevamente a lista, y si el DF está vacío generar una lista de 300 ceros
#Que inserte la lista de vectores del documento en la lista de vectores del conjunto
#Que convierta la lsita de vectores del set (que a su vez contiene las listas de vectores de cada documento) en np_array


#Se recomienda hacer los append a listas y luego convertir la lista de listas en array
#https://stackoverflow.com/questions/31250129/python-numpy-array-of-numpy-arrays

def mean_embedded_vectors(type_of_set):
    set_vectors_list = []

    i = 0
    for doc in type_of_set:
        vectors = [midicc_kv[keyword] for keyword in doc]
        df_vectors = pd.DataFrame(vectors).transpose()
        df_vectors_mean = df_vectors.mean(axis=1)
        df_vectors_mean_ls = df_vectors_mean.tolist()
        if len(df_vectors_mean_ls) == 0:
            print("La longitud del elemento: " + str(i) + "    es: " + str(len(df_vectors_mean_ls)))
            df_vectors_mean_ls = [0] * n
        set_vectors_list.append(df_vectors_mean_ls)
        i = i + 1

    return np.asarray(set_vectors_list)

print("Se genera Train-Emb:")
trainEmb = mean_embedded_vectors(train_x)

print("Se genera Val-Emb:")
valEmb = mean_embedded_vectors(val_x)

print("Se genera Test-Emb:")
testEmb = mean_embedded_vectors(test_x)


# *********** Aquí termina la sección de agregar código *************

Se genera Train-Emb:
La longitud del elemento: 657    es: 0
La longitud del elemento: 967    es: 0
La longitud del elemento: 1550    es: 0
Se genera Val-Emb:
La longitud del elemento: 302    es: 0
Se genera Test-Emb:
La longitud del elemento: 258    es: 0
La longitud del elemento: 342    es: 0


In [15]:
# Veamos las dimensiones de cada conjunto embebido:

print("Train-Emb:", trainEmb.shape)
print("Val-Emb:", valEmb.shape)
print("Test-Emb:", testEmb.shape)

Train-Emb: (2100, 300)
Val-Emb: (450, 300)
Test-Emb: (450, 300)


# **Pregunta - 8:**



Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños.

Compara los resultados con los de la semana anterior.

In [16]:
# REGRESIÓN LOGÍSTICA:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


modeloLR = LogisticRegression(max_iter=10000, C=4, solver='saga', penalty='l1')
modeloLR.fit(trainEmb, y_train)

print('LR: Train-accuracy: %.2f%%' % (100*modeloLR.score(trainEmb, y_train)))
print('LR: Val-accuracy: %2.f%%' % (100*modeloLR.score(valEmb, y_val)))

# *********** Aquí termina la sección de agregar código *************


LR: Train-accuracy: 83.71%
LR: Val-accuracy: 82%


In [79]:
# BOSQUE ALEATORIO (Random Forest):

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

'''
modeloRF = RandomForestClassifier(n_estimators =900,
 max_samples = 28,
 min_samples_split = 5,
 min_samples_leaf = 1,
 max_depth = 3,
 bootstrap = True,
 criterion = 'entropy',
 max_features = None,
 #max_leaf_nodes = 3,
 n_jobs = -1)

'''
modeloRF = RandomForestClassifier(n_estimators = 200,
 max_samples = 45,
 min_samples_split = 5,
 min_samples_leaf = 1,
 max_depth = 5,
 bootstrap = True,
 criterion = 'entropy',
 max_features = None,
 #max_leaf_nodes = 3,
 n_jobs = -1)



modeloRF.fit(trainEmb, y_train)

print('\nRF: Train-accuracy: %.2f%%' % (100*modeloRF.score(trainEmb, y_train)))
print('RF: Val-accuracy: %.2f%%' % (100*modeloRF.score(valEmb, y_val)))

# *********** Aquí termina la sección de agregar código *************


RF: Train-accuracy: 75.52%
RF: Val-accuracy: 72.67%


#**Pregunta - 9:**



Reporte del mejor modelo.


In [80]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

mejor_modelo = modeloLR


print('Test-accuracy con el mejor modelo %.2f%%' % (100*mejor_modelo.score(testEmb, y_test)))

from sklearn.metrics import confusion_matrix

pred = mejor_modelo.predict(testEmb)
print('\nMatriz de confusión con el mejor modelo de Conteo:')
print(confusion_matrix(y_test, pred, labels=[0,1]))

print('\nMatriz de confusión con el mejor modelo de Conteo en proporciones:')
print(confusion_matrix(y_test, pred, labels=[0,1]) / pred.shape[0])


# *********** Aquí termina la sección de agregar código *************

Test-accuracy con el mejor modelo 80.44%

Matriz de confusión con el mejor modelo de Conteo:
[[172  44]
 [ 44 190]]

Matriz de confusión con el mejor modelo de Conteo en proporciones:
[[0.38222222 0.09777778]
 [0.09777778 0.42222222]]


#**Pregunta - 10:**



Incluye tus comentarios finales de la actividad.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++

##**Fin de la Actividad de vectores Embebidos - FastText**

In [165]:
#Randomized Search


# Number of trees in random forest
n_estimators = [500, 800, 1000, 1200, 1500]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2', None]
# Maximum number of levels in tree
max_depth = [3,4,5,6,7]
#max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3, 5, 8]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': ['log_loss']}

print(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=4, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(trainEmb, y_train)

rf_random.best_params_




{'n_estimators': [500, 800, 1000, 1200, 1500], 'max_features': ['auto', 'sqrt', 'log2', None], 'max_depth': [3, 4, 5, 6, 7], 'min_samples_split': [2, 3, 5, 8], 'min_samples_leaf': [1, 2, 3, 4], 'bootstrap': [True, False], 'criterion': ['log_loss']}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


{'n_estimators': 1200,
 'min_samples_split': 8,
 'min_samples_leaf': 1,
 'max_features': None,
 'max_depth': 7,
 'criterion': 'log_loss',
 'bootstrap': True}